In [1]:
from sentence_transformers import SentenceTransformer
import requests
import numpy as np
from tqdm import tqdm
import pandas as pd
from elasticsearch import Elasticsearch

embeding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

/usr/local/python/3.10.13/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
user_question = "I just discovered the course. Can I still join it?"
embedding = embeding_model.encode(user_question)
print(embedding[0])

0.078222655


In [3]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

print(len(documents))

948


In [4]:
print(documents[0])

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp', 'id': 'c02e79ef'}


In [5]:
course = 'machine-learning-zoomcamp'
ml_documents = [item for item in documents if item['course'] == course]
print(len(ml_documents))

375


In [6]:
embeddings = list()
for ml_doc in tqdm(ml_documents):
    qa_text = f"{ml_doc['question']} {ml_doc['text']}"
    embeddings.append(embeding_model.encode(qa_text))

X = np.array(embeddings)
print(X.shape)

  0%|          | 0/375 [00:00<?, ?it/s]

100%|██████████| 375/375 [00:36<00:00, 10.27it/s]

(375, 768)


In [7]:
embedding_score = embedding.dot(embedding)
print(embedding_score)

0.9999999


In [8]:
scores = X.dot(embedding)
print(scores.max())

0.6506573


In [9]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings
        
    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argpartition(-scores, num_results - 1)[:num_results]
        return [self.documents[i] for i in idx]
    
search_engine = VectorSearchEngine(ml_documents, X)
search_engine.search(embedding, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [10]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [11]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        embedded_question = embeding_model.encode(q['question'])
        results = search_function(embedded_question)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [12]:
evaluate(ground_truth, search_engine.search)

  0%|          | 0/1830 [00:00<?, ?it/s]

100%|██████████| 1830/1830 [02:16<00:00, 13.42it/s]


{'hit_rate': 0.9579234972677596, 'mrr': 0.4372842397432573}

In [19]:
es_client = Elasticsearch('http://localhost:9200')

In [20]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": len(embeddings[0]),
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True, timeout="2m", master_timeout="2m")
es_client.indices.create(index=index_name, body=index_settings, timeout="2m", master_timeout="2m")

ConnectionTimeout: Connection timed out

In [21]:
for doc in tqdm(ml_documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = embeding_model.encode(question)
    doc['text_vector'] = embeding_model.encode(text)
    doc['question_text_vector'] = embeding_model.encode(qt)

100%|██████████| 375/375 [01:21<00:00,  4.63it/s]


In [22]:
for doc in tqdm(ml_documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:10<?, ?it/s]


ConnectionTimeout: Connection timed out